In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# 1. Download all the data from AEI website

In [3]:
# Desactivar la verificación del certificado SSL
requests.packages.urllib3.disable_warnings()

# URL de la página
url = "https://www.aei.gob.es/ayudas-concedidas/buscador-ayudas-concedidas?year={}&convocatory=All&code=&gender=All&area=&subarea=&title=&keywords=&cif=&name=&ccaa=All&province=All&granted%5Bmin%5D=&granted%5Bmax%5D=&summary="
years =  ["2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]

for year in years:
    # Realizar la solicitud GET a la página sin verificar el certificado SSL
    response = requests.get(url.format(year), verify=False)

    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Analizar el HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar el botón de descarga del archivo CSV
        download_button = soup.find('a', class_='btn-primary', title='Descargar archivo')

        if download_button:
            # Obtener la URL de descarga
            csv_url = "https://www.aei.gob.es" + download_button['href']

            # Descargar el archivo CSV
            csv_response = requests.get(csv_url, verify=False)

            # Verificar si la descarga fue exitosa
            if csv_response.status_code == 200:
                # Guardar el contenido en un archivo local
                with open('raw_data/datos_aei_{}.csv'.format(year), 'wb') as f:
                    f.write(csv_response.content)
                print("Los datos del año {} se han descargado correctamente.".format(year))
            else:
                print("No se pudo descargar el archivo CSV del año {}.".format(year))
        else:
            print("No se encontró el botón de descarga del CSV del año {}.".format(year))
    else:
        print("Error al acceder a la página:", response.status_code)


Los datos del año 2008 se han descargado correctamente.
Los datos del año 2009 se han descargado correctamente.
Los datos del año 2010 se han descargado correctamente.
Los datos del año 2011 se han descargado correctamente.
Los datos del año 2012 se han descargado correctamente.
Los datos del año 2013 se han descargado correctamente.
Los datos del año 2014 se han descargado correctamente.
Los datos del año 2015 se han descargado correctamente.
Los datos del año 2016 se han descargado correctamente.
Los datos del año 2017 se han descargado correctamente.
Los datos del año 2018 se han descargado correctamente.
Los datos del año 2019 se han descargado correctamente.
Los datos del año 2020 se han descargado correctamente.
Los datos del año 2021 se han descargado correctamente.
Los datos del año 2022 se han descargado correctamente.
Los datos del año 2023 se han descargado correctamente.


# 2. Compose the AEI Dataset

In [4]:
for year in years:
    df = pd.read_csv('/export/usuarios_ml4ds/mbalairon/github/data_ingest/AEI/raw_data/datos_aei_{}.csv'.format(year), sep=';')
    print('De las', df.shape[0], 'entradas de {},'.format(year), df['Resumen'].isna().sum(), 'no tienen resumen.')

De las 5534 entradas de 2008, 2099 no tienen resumen.
De las 6884 entradas de 2009, 3047 no tienen resumen.
De las 7351 entradas de 2010, 3895 no tienen resumen.
De las 9175 entradas de 2011, 5740 no tienen resumen.
De las 6873 entradas de 2012, 3600 no tienen resumen.
De las 6183 entradas de 2013, 2858 no tienen resumen.
De las 8151 entradas de 2014, 5165 no tienen resumen.
De las 7951 entradas de 2015, 3555 no tienen resumen.
De las 6315 entradas de 2016, 3223 no tienen resumen.
De las 5968 entradas de 2017, 2604 no tienen resumen.
De las 7461 entradas de 2018, 3423 no tienen resumen.
De las 6377 entradas de 2019, 2403 no tienen resumen.
De las 5993 entradas de 2020, 2517 no tienen resumen.
De las 8668 entradas de 2021, 2743 no tienen resumen.
De las 8614 entradas de 2022, 3951 no tienen resumen.
De las 292 entradas de 2023, 0 no tienen resumen.


In [5]:
df = pd.DataFrame()
for year in years:
    df_aux = pd.read_csv('/export/usuarios_ml4ds/mbalairon/github/data_ingest/AEI/raw_data/datos_aei_{}.csv'.format(year), sep=';')
    df = pd.concat([df, df_aux])

In [6]:
df = df.rename(columns={'€ Conced.': 'presupuesto_concedido'})

In [7]:
df.head()

,Año,Convocatoria,Referencia,Género,Área,Subárea,Título,C.I.F.,Entidad,CC.AA.,Provincia,presupuesto_concedido,Resumen
0,2008,CONSOLIDER,CSD2008-00005,NaN,NaN,NaN,NaN,Q5350015C,UNIVERSIDAD MIGUEL HERNANDEZ DE ELCHE,C.VALENCIANA,ALICANTE,"1.144.400,00",NaN
1,2008,CONSOLIDER,CSD2008-00005,NaN,NaN,NaN,NaN,Q5350015C,UNIVERSIDAD MIGUEL HERNANDEZ DE ELCHE,C.VALENCIANA,ALICANTE,"184.000,00",NaN
2,2008,CONSOLIDER,CSD2008-00005,NaN,NaN,NaN,NaN,Q3318001I,UNIVERSIDAD DE OVIEDO,PDO.ASTURIAS,ASTURIAS,"186.000,00",NaN
3,2008,CONSOLIDER,CSD2008-00005,NaN,NaN,NaN,NaN,G83726984,FUNDACION PARA LA INVESTIGACION BIOMEDICA DEL ...,MADRID,MADRID,"224.000,00",NaN
4,2008,CONSOLIDER,CSD2008-00005,NaN,NaN,NaN,NaN,G46923421,FUNDACIÓN DE LA COMUNIDAD VALENCIANA CENTRO DE...,C.VALENCIANA,VALENCIA,"927.000,00",NaN


In [8]:
df.to_parquet('/export/usuarios_ml4ds/mbalairon/github/data_ingest/AEI/output/datos_aei.parquet', index=False)

In [10]:
df.to_parquet('/export/data_ml4ds/AI4U/Datasets/AEI/20240321/datos_aei.parquet', index=False)

In [13]:
df[df['Entidad']=='UNIVERSIDAD CARLOS III DE MADRID']

,Año,Convocatoria,Referencia,Género,Área,Subárea,Título,C.I.F.,Entidad,CC.AA.,Provincia,presupuesto_concedido,Resumen
24,2008,CONSOLIDER,CSD2008-00007,NaN,NaN,NaN,NaN,Q2818029G,UNIVERSIDAD CARLOS III DE MADRID,MADRID,MADRID,"1.336.832,56",NaN
35,2008,CONSOLIDER,CSD2008-00007,NaN,NaN,NaN,NaN,Q2818029G,UNIVERSIDAD CARLOS III DE MADRID,MADRID,MADRID,"3.500.000,00",NaN
57,2008,CONSOLIDER,CSD2008-00010,NaN,NaN,NaN,NaN,Q2818029G,UNIVERSIDAD CARLOS III DE MADRID,MADRID,MADRID,"511.200,00",NaN
79,2008,CONSOLIDER,CSD2008-00077,NaN,NaN,NaN,NaN,Q2818029G,UNIVERSIDAD CARLOS III DE MADRID,MADRID,MADRID,"6.128,16",NaN
320,2008,Ramón y Cajal,RYC-2008-03371,HOMBRE,NaN,NaN,"Contratacion multipartidaria, toma de decision...",Q2818029G,UNIVERSIDAD CARLOS III DE MADRID,MADRID,MADRID,"192.480,00",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8559,2022,Juan de la Cierva,JDC2022-050288-I,HOMBRE,Ciencias y tecnologías de materiales,Materiales para la energía y el medioambiente,High-Entropy Ceramics based on Laminate Metal ...,Q2818029G,UNIVERSIDAD CARLOS III DE MADRID,MADRID,MADRID,"67.400,00",NaN
38,2023,Proyectos de Colaboración Internacional,PCI2023-143425,HOMBRE,Energía y transporte,Transporte,"INTELLIGENT, SAFE AND SECURE CONNECTED ELECTRI...",Q2818029G,UNIVERSIDAD CARLOS III DE MADRID,MADRID,MADRID,"216.590,00",ECOMOBILITY APOYARA A LA INDUSTRIA Y LAS CIUDA...
103,2023,Ayudas ATRAE para la incorporación de talento ...,ATR2023-145754,NaN,Tecnologías de la información y de las comunic...,Ciencias de la computación y tecnología inform...,DESENTRAÑAR LAS REDES NEURONALES PROFUNDAS GEN...,Q2818029G,UNIVERSIDAD CARLOS III DE MADRID,MADRID,MADRID,"971.575,00","EN UGAI, NOS CENTRAREMOS EN CONTROLAR MODELOS ..."
110,2023,Ayudas ATRAE para la incorporación de talento ...,ATR2023-145734,NaN,Economía,"Economía, métodos y aplicaciones",REASIGNACION LABORAL POR UN FUTURO SOSTENIBLE ...,Q2818029G,UNIVERSIDAD CARLOS III DE MADRID,MADRID,MADRID,"777.325,00","LA APARICION DE NUEVAS TECNOLOGIAS, EL ENVEJEC..."
